In [ ]:
import os
import sys
import numpy as np
import scanpy as sc
import torch
import nichecompass

prjdir = os.path.abspath(os.path.join(os.getcwd(), '../..'))
if prjdir not in sys.path:
    sys.path.append(prjdir)

sc.settings.n_jobs=40

In [ ]:
%load_ext autoreload
%autoreload 2
import src.spatial_helpers.spatial as spp
import src.spatial_helpers.sc as scp
scp.set_all_seeds()

In [ ]:
datadir = '../../data/xenium_merged'
resultsdir = os.path.join(datadir, 'nichecompass')
ncdir = os.path.join(datadir, 'nichecompass')
os.makedirs(resultsdir, exist_ok=True)
sc.settings.figdir = os.path.join(resultsdir)

## NicheCompass

In [ ]:
# params
infile = os.path.join(datadir, 'crc_ffpe_norm.h5ad')
batch_key = 'name'
layer_key =  'counts'
gpdir = os.path.join(ncdir, 'delaunayr50')
key = 'delaunayr50'
adj_key = 'delaunayr50_connectivities' 
n_neighbors = 15
n_iterations = 5
resolutions = [float(x) for x in '0.05,0.1,0.15,0.2'.split(',')]
encoder = 'gcnconv'

In [ ]:
adata = sc.read_h5ad(infile)
adata.obsp['spatial_connectivities'] = adata.obsp[adj_key].copy()
adata.X = adata.layers[layer_key]
gps = spp.nc_prepare_gps(dir=gpdir)
model = spp.nc_run(adata, gps, batch_keys=[batch_key], layer=layer_key, adj_key=adj_key, latent_dtype=np.float64, conv_layer_encoder=encoder)
model.save(dir_path=os.path.join(ncdir, key, 'model'), overwrite=True, save_adata=True, adata_file_name='adata.h5ad')

In [ ]:
adata = sc.read_h5ad(os.path.join(ncdir, key, 'model', 'adata.h5ad'))
adata.obsm[key] = adata.obsm['nichecompass_latent'].copy()
del adata.obsm['nichecompass_latent']
sc.pp.neighbors(adata, use_rep=key, n_neighbors=n_neighbors, key_added=key+'_nb')
for res in resolutions:
    sc.tl.leiden(adata, resolution=res, neighbors_key=key+'_nb', key_added=key+'_nb_leiden_'+str(res), n_iterations=n_iterations)
adata = scp.rename_clusters_by_size(adata, cluster_key=key+'_nb_leiden_'+str(res))
adata.write_h5ad(os.path.join(ncdir, key, 'model', 'adata.h5ad'))